# Impact of Trading Conditions

## I - Presentation
###   I.1 - Overview
Intraday or Ticks financial data can be visualized using plenty different manner (graphs, apple_bars, etc…). These visualisations will mostly focus on the size or the price of the trades on the timeline studied.

But what about the trading conditions of these trades ? Is there a difference if we study one or all trading conditions ?

This sample will demonstrate the impact of these trading conditions on the visualization of market ticks data.
First it will show the difference of trading conditions between two exchanges (Nasdaq and EuroNext Paris).
Then a specific study of the 'Nasdaq 100' exchange and the Apple 'AAPL' ticker will allow us to observe the proportion of each type of trades.
Finally it will try to observe ticks bars graphs of the 'AAPL' ticker and determine if accepting or rejecting parts of the trading conditions impact the analysis of the bars graphs.

###   I.2 - Services Used
This sample uses *gRPC requests* in order to retrieve data from the dedicated hosted service. The queried endpoints in this script are:
* *TickConditionsService*: used to get the raw data of the trading conditions informations of a specific exchange
* *TickTradeConditionService*: to retrieve the trades conditions data (quantity, volume, percentage) of an exchange or a ticker 
* *TickBarsService*: to directly retrieve tick bars objects from the server.
* *StaticDataService*: to retrieve reference data objects from the server (components and weights for a given exchange in our case)

###   I.3 Modules Required
1. Systemathics:
    * *systemathics.apis.services.tick_conditions.v1*
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.services.tick_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *systemathics.apis.services.static_data.v1*
    * *google.type*
    
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly*, *matpotlib* and *mplfinance* as per display packages
    
###   I.4 Notebook Prepartion
#### Packages and Import


In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly mplfinance matplotlib

In [ ]:
pip install systemathics.apis --pre

In [ ]:
import os
import grpc
import numpy as np
import pandas as pd
import mplfinance as mpf
import plotly.express as px
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.indices.v1.components_pb2 as indices
import systemathics.apis.services.indices.v1.components_pb2_grpc as indices_service
import systemathics.apis.services.tick_conditions.v1.tick_conditions_pb2 as tick_conditions
import systemathics.apis.services.tick_conditions.v1.tick_conditions_pb2_grpc as tick_conditions_service
import systemathics.apis.services.tick_analytics.v1.tick_trade_conditions_pb2 as tick_trade_conditions
import systemathics.apis.services.tick_analytics.v1.tick_trade_conditions_pb2_grpc as tick_trade_conditions_service
import systemathics.apis.services.tick_analytics.v1.tick_trade_condition_statistics_pb2 as tick_trade_condition_statistics
import systemathics.apis.services.tick_analytics.v1.tick_trade_condition_statistics_pb2_grpc as tick_trade_condition_statistics_service
import systemathics.apis.services.tick_analytics.v1.tick_vwap_pb2 as tick_vwap
import systemathics.apis.services.tick_analytics.v1.tick_vwap_pb2_grpc as tick_vwap_service
import systemathics.apis.services.tick_analytics.v1.tick_bars_pb2 as tick_bars
import systemathics.apis.services.tick_analytics.v1.tick_bars_pb2_grpc as tick_bars_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

#### Retrieve Authentification token

In [ ]:
token = token_helpers.get_token()
display(token)

***
## II - Trade condition
###   II.1 - Definition
A trade occurs when a buyer (bid side) and a seller (ask side) match their interest on the market. The unique identity of the trade is given by at least six properties:
•	The counterparts: buyer and seller
•	The execution unique identifier
•	The timestamp of trade
•	The execution price
•	The size: total executed shares
•	The trading condition

More notably, the trading condition is determined by the market and represents how the trade happened and its type. The most common trading conditions are “Regular Trade”, “Auction Trade”, and “Over The Counter (OTC)”.

###   II.2 - Un-normalized trading conditions between financial markets
Even if all marketplaces used these trading conditions, there isn’t a common definition between them. Each Marketplace has their own trading conditions, name and value.

Let’s study XNGS and XPAR exchanges trading conditions :


#### Retrieve trade conditions

In [ ]:
# set inputs
exchange_xngs = 'XNGS'
exchange_xpar = 'XPAR'

In [ ]:
# generate tick conditions request
request_xngs = tick_conditions.TickConditionsRequest(exchange = exchange_xngs)
request_xpar = tick_conditions.TickConditionsRequest(exchange = exchange_xpar)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick condition service
        service = tick_conditions_service.TickConditionsServiceStub(channel)
        
        # process the tick condition requests
        response_xngs = service.TickConditions(
            request=request_xngs, 
            metadata = [('authorization', token)]
        )
        response_xpar = service.TickConditions(
            request=request_xpar, 
            metadata = [('authorization', token)]
        )
        
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

DataFrame creation

In [ ]:
# Retrieve data from resquests responses
# for xngs
value_xngs = [d.value for d in response_xngs.data]
description_xngs = [d.description for d in response_xngs.data]
comment_xngs = [d.comment for d in response_xngs.data]
isregular_xngs = [d.regular for d in response_xngs.data]
isauction_xngs = [d.auction for d in response_xngs.data]

# for xpar
value_xpar = [d.value for d in response_xpar.data]
description_xpar = [d.description for d in response_xpar.data]
comment_xpar = [d.comment for d in response_xpar.data]
isregular_xpar = [d.regular for d in response_xpar.data]
isauction_xpar = [d.auction for d in response_xpar.data]
    
# Create the corresponding pandas dataframe
d_xngs = {'Value': value_xngs, 'Description': description_xngs, 'Comment': comment_xngs, 'IsRegular': isregular_xngs, 'IsAuction': isauction_xngs}
df_xngs = pd.DataFrame(data=d_xngs)

d_xpar = {'Value': value_xpar, 'Description': description_xpar, 'Comment': comment_xpar, 'IsRegular': isregular_xpar, 'IsAuction': isauction_xpar}
df_xpar = pd.DataFrame(data=d_xpar)

#### XNGS Trading Conditions

In [ ]:
df_xngs.style.set_table_attributes("style='display:inline'").set_caption(exchange_xngs + " Trading Conditions")

#### XPAR Trading Conditions

In [ ]:
df_xpar.style.set_table_attributes("style='display:inline'").set_caption(exchange_xpar + " Trading Conditions")

Some observations can be made : 
-	XNGS has 34 trading conditions with value between 0 and 57 whereas XPAR has only 20 trading conditions with value between 0 and 23
-	If we choose the ‘Cross trade’ trading conditions, on XNGS it has the value 24 and on XPAR the value 2.
-	The value 17 is corresponding to the 'Market Center Open Price' trade for XNGS and 'Odd Lot' Trades for XPAR.

These observations are proof that one need to be careful of these differences when using trading conditions.
After a close study of these trading confitions, there may be similiraties between some of them. This could probably allow one to build clusters of trading conditions.

***
We have demonstrated that trading conditions are different between exchanges.
Now we are going to study one exchange and the proportion of each trading conditions.
***

## III - Proportion of trading condition - Study on the Nasdaq 100 exchange
After one got the trading conditions of the studied market, it is preferrable to find the proportion of each one of them in the market, and for each equity of the market.

Let’s study the XNGS exchanges group, especialy the 'Nasdaq 100' exchange.

First we need to get the 'Nasdaq 100' components. We are retrieving them with a request allowing us to have the weight of each equity in the exchange.

### Step 1: Create and process request

In [ ]:
# which market place we want to study
market_place = 'nasdaq 100'

# generate the indices request
request_nsdq = indices.ComponentsRequest( 
    identifier = market_place
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the indices service
        service = indices_service.ComponentsServiceStub(channel)
        
        # process the indices request
        response_nsdq = service.Components(
            request = request_nsdq, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

### Step 2 Retrieve components and weights

In [ ]:
# retrieve data from resquest response
tickers = [c.identifier.ticker for c in response_nsdq.components]
exchanges =[c.identifier.exchange for c in response_nsdq.components]
weights = [c.weight for c in response_nsdq.components]

# create the dataframe
d = {'Ticker': tickers, 'Exchange': exchanges, 'Weight': weights }
NSD = pd.DataFrame(data=d)
NSD = NSD.sort_values(by = 'Weight')
NSD.sort_values(by=['Weight'], inplace=True, ascending = False)
NSD.head(10)

Now that's we have the components, we can retrieves the trades condtitions datas for each one of them

### Step 3 Retreiving Trading Conditions Data
#### Token Selection

In [ ]:
# generate the tick conditions request for the different assets
my_identifiers = []
for row in NSD.itertuples():
    my_identifier = identifier.Identifier(exchange = row.Exchange, ticker = row.Ticker)
    my_identifiers.append(my_identifier)

#### Time period selection
Note that we will keep this time interval for the entire notebook

In [ ]:
# create time intervals (we are using Google date format)
today = datetime.today()
my_days = 100 # time interval of study
start = today - timedelta(days=my_days)

date_interval_nsdq = dateinterval.DateInterval(
    start_date = date.Date(year = 2022, month = 2, day = 9), 
    end_date = date.Date(year = 2022, month = 5, day = 22)
)

#### Request

In [ ]:
# generate constraints based on the previous time selection
constraints_nsdq = constraints.Constraints(
    date_intervals = [date_interval_nsdq]
)

Tick trades conditions request

In [ ]:
%%time
# generate the tick trades request for all the nasdaq 100 tickers
xngs_trades = {}
for id in my_identifiers:
    
    request_tcs = tick_trade_condition_statistics.TickTradeConditionStatisticsRequest(
        identifier = id,
        constraints = constraints_nsdq
    )

    try:
        # open a gRPC channel
        with channel_helpers.get_grpc_channel() as channel:  
        
            # instantiate the tick apple_bars service
            service = tick_trade_condition_statistics_service.TickTradeConditionStatisticsServiceStub(channel)
        
            # process the tick apple_bars request
            statistics = []
            my_metadata = [('authorization', token)]
            for stat in service.TickTradeConditionStatistics(request=request_tcs, metadata=my_metadata):
                statistics.append(stat)
    except grpc.RpcError as e:
        display(e.code().name)
        display(e.details())
        display(id)
    
    xngs_trades[id.ticker] = statistics


### Step 4 Calculating Trading Conditions proportions

#### Trading Conditions Quantities based on trades volumes

In [ ]:
TC_QuantityV = df_xngs.iloc[:, 0:2].copy()
for xngs_ticker in xngs_trades: #Trading conditions from Nsdq 100
    for tc in range(len(xngs_trades[xngs_ticker])):
        TC_QuantityV.loc[TC_QuantityV.Value == xngs_trades[xngs_ticker][tc].value, xngs_ticker] = int(xngs_trades[xngs_ticker][tc].volume)
TC_QuantityV['Total'] = TC_QuantityV.iloc[:, 2:].sum(axis=1) #give the quantity of each trades conditions for all the exchange
TC_QuantityV= TC_QuantityV[TC_QuantityV['Total'] != 0].fillna(0)
TC_QuantityV.style.set_table_attributes("style='display:inline'").set_caption("Trading Conditions Quantities (nb of trades)").format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(TC_QuantityV.iloc[:,2:-1].max()), cmap="Blues", subset=TC_QuantityV.columns[2:-1])

#### Trading Conditions Quantities based on trades counts

In [ ]:
TC_QuantityC = df_xngs.iloc[:, 0:2].copy()
for xngs_ticker in xngs_trades: #Trading conditions from Nsdq 100
    for tc in range(len(xngs_trades[xngs_ticker])):
        TC_QuantityC.loc[TC_QuantityC.Value == xngs_trades[xngs_ticker][tc].value, xngs_ticker] = int(xngs_trades[xngs_ticker][tc].count)
TC_QuantityC['Total'] = TC_QuantityC.iloc[:, 2:].sum(axis=1) #give the quantity of each trades conditions for all the exchange
TC_QuantityC= TC_QuantityC[TC_QuantityC['Total'] != 0].fillna(0)
TC_QuantityC.style.set_table_attributes("style='display:inline'").format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(TC_QuantityC.iloc[:,2:-1].max()), cmap="Blues", subset=TC_QuantityC.columns[2:-1]).set_caption("Trading Conditions Quantities (nb of trades)")

### Step 5 Visualisation of the trading conditions distributions

In [ ]:
fig = px.pie(TC_QuantityC.sort_index(), values='Total', names='Description',
            title = ('Trading Conditions Count Proportion for {0} ({1} trades between {2} and {3})').format('Nasdaq 100',round(TC_QuantityC['Total'].sum(),0),start.date(),today.date()),
            color='Value')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide', width=1500, height=500)
fig.show()

In [ ]:
fig = px.pie(TC_QuantityV.sort_index(), values='Total', names='Description',
            title = ('Trading Conditions Volumes Proportion for {0} ({1} trades between {2} and {3})').format('Nasdaq 100',round(TC_QuantityV['Total'].sum(),0),start.date(),today.date()),
            color='Value')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide', width=1500, height=500)
fig.show()

Observations : 
- 3 mainly used trading conditions (Odd Lot, Regular sale, Intermarketsweep) for 99.6% of the trades in the exchange but for nearly 84% of the total volume traded
- Odd Lot Trade : 72.2% of the trades but only 18.1% of the volume traded
- Closing Prints and Form T : less than 0.5 % of the trades but more than 12% of the volume traded

### Export / Import
Export

In [ ]:
#TC_QuantityC.to_csv(('TC_QuantityC({0}d).csv').format(my_days), index=True)
#TC_QuantityV.to_csv(('TC_QuantityV({0}d).csv').format(my_days), index=True)

Import
if you already have exported csv files

In [ ]:
#TC_QuantityC = pd.read_csv (r'TC_QuantityC(*d).csv')
#TC_QuantityV = pd.read_csv (r'TC_QuantityV(*d).csv')

***

We have analyse the Nasdaq 100 behaviour in front of the trading conditions. The next step is to determine if the same analysis can be made for the tickers in the Nasdaq 100.
For that we are now going to study the biggest ticker, by far Apple - 'AAPL', for a macroscopic view, and the one of the smallest for a microscopic view.

***


## IV - Daily study



In [ ]:
# create time intervals (we are using Google date format)
today = datetime.today()
my_days = 100
start = today - timedelta(days=my_days)

In [ ]:
date_interval_daily = dateinterval.DateInterval(
    start_date = date.Date(year = 2022, month = 2, day = 18), 
    end_date = date.Date(year = 2022, month = 5, day = 19)
)

my_constraints_daily = constraints.Constraints(
    date_intervals = [date_interval_daily],
    excluded_days = [dayofweek.SUNDAY, dayofweek.SATURDAY]
)

### Apple (AAPL)
Let's analyse Apple trades behaviour in front of the trading conditions.

#### Request

In [ ]:
request_daily = tick_trade_conditions.TickTradeConditionsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_daily,
    sampling = duration.Duration(seconds = 1 * 60 * 60 * 24),
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick apple_bars service
        service = tick_trade_conditions_service.TickTradeConditionsServiceStub(channel)
        
        # process the tick apple_bars request
        apple_daily = []
        my_metadata = [('authorization', token)]
        for stat in service.TickTradeConditions(request=request_daily, metadata=my_metadata):
            apple_daily.append(stat)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Percentage of Trading Condition - Daily

##### Percentage based on trades counts

DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in apple_daily]
d = {'Date': dates}
df_apple_daily_proC= pd.DataFrame(data = d)

In [ ]:
for daily in  apple_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_apple_daily_proC:
            df_apple_daily_proC.iat[df_apple_daily_proC.index[df_apple_daily_proC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_apple_daily_proC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].percentage * 100
        else:
            df_apple_daily_proC[str(daily.data[index].value)] = 0.0
            df_apple_daily_proC.iat[df_apple_daily_proC.index[df_apple_daily_proC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_apple_daily_proC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].percentage * 100
            
df_apple_daily_proC = df_apple_daily_proC.set_index('Date')          

In [ ]:
df_apple_daily_proC = df_apple_daily_proC[(df_apple_daily_proC.T != 0.).any()]
df_apple_daily_proC.style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_apple_daily_proC.max()), cmap="Blues")

##### Percentage based on trades volumes

DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in apple_daily]
d = {'Date': dates}
df_apple_daily_proV= pd.DataFrame(data = d)

In [ ]:
for daily in  apple_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_apple_daily_proV:
            df_apple_daily_proV.iat[df_apple_daily_proV.index[df_apple_daily_proV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_apple_daily_proV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume_percentage * 100
        else:
            df_apple_daily_proV[str(daily.data[index].value)] = 0.0
            df_apple_daily_proV.iat[df_apple_daily_proV.index[df_apple_daily_proV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_apple_daily_proV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume_percentage * 100
            
df_apple_daily_proV = df_apple_daily_proV.set_index('Date')          

In [ ]:
df_apple_daily_proV = df_apple_daily_proV[(df_apple_daily_proV.T != 0.).any()]
df_apple_daily_proV.style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_apple_daily_proV.max()), cmap="Blues")

##### Visualisation 

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_apple_daily_proV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_apple_daily_proV[col], label=lab)
        axC.plot(df_apple_daily_proC[col])
        
fig.suptitle(('Trading Conditions Daily Proportions of Apple (AAPL-XNGS) for {0} days').format(my_days), fontsize=20)

axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count (%)",color="blue",fontsize=14)

axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume (%)",color="blue",fontsize=14)

# Highlight weekends based on the x-axis units
xmin, xmax = axC.get_xlim()
days = np.arange(np.floor(xmin), np.ceil(xmax)+2) # range of days in date units
weekends = [(dt.weekday()>=5)|(dt.weekday()==0) for dt in mdates.num2date(days)]
axC.fill_between(days, *axC.get_ylim(), where=weekends, alpha=.1)
axC.set_xlim(xmin, xmax) # set limits back to default values
axV.fill_between(days, *axV.get_ylim(), where=weekends, alpha=.1)
axV.set_xlim(xmin, xmax) # set limits back to default values

fig.legend()
fig.show()

#### Quantities of Trading Conditions - Daily

##### Quantities based on trades counts

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in apple_daily]
d = {'Date': dates}
df_apple_daily_quaC= pd.DataFrame(data = d)

In [ ]:
for daily in  apple_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_apple_daily_quaC:
            df_apple_daily_quaC.iat[df_apple_daily_quaC.index[df_apple_daily_quaC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_apple_daily_quaC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].count
        else:
            df_apple_daily_quaC[str(daily.data[index].value)] = 0.0
            df_apple_daily_quaC.iat[df_apple_daily_quaC.index[df_apple_daily_quaC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_apple_daily_quaC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].count
            
df_apple_daily_quaC = df_apple_daily_quaC.set_index('Date')          

In [ ]:
df_apple_daily_quaC = df_apple_daily_quaC[(df_apple_daily_quaC.T != 0.).any()]
df_apple_daily_quaC.style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_apple_daily_quaC.max()), cmap="Blues")

##### Quantities based on trades volumes

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in apple_daily]
d = {'Date': dates}
df_apple_daily_quaV= pd.DataFrame(data = d)

In [ ]:
for daily in  apple_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_apple_daily_quaV:
            df_apple_daily_quaV.iat[df_apple_daily_quaV.index[df_apple_daily_quaV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_apple_daily_quaV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume
        else:
            df_apple_daily_quaV[str(daily.data[index].value)] = 0.0
            df_apple_daily_quaV.iat[df_apple_daily_quaV.index[df_apple_daily_quaV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_apple_daily_quaV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume
            
df_apple_daily_quaV = df_apple_daily_quaV.set_index('Date')          

In [ ]:
df_apple_daily_quaV = df_apple_daily_quaV[(df_apple_daily_quaV.T != 0.).any()]
df_apple_daily_quaV.style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_apple_daily_quaV.max()), cmap="Blues")

##### Visualisation

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_apple_daily_quaV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_apple_daily_quaV[col], label=lab)
        axC.plot(df_apple_daily_quaC[col])
        
fig.suptitle('Trading Conditions Daily Quantities of Apple (AAPL-XNGS) 100 days', fontsize=20)

axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count",color="blue",fontsize=14)

axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume",color="blue",fontsize=14)

# Highlight weekends based on the x-axis units
xmin, xmax = axC.get_xlim()
days = np.arange(np.floor(xmin), np.ceil(xmax)+2) # range of days in date units
weekends = [(dt.weekday()>=5)|(dt.weekday()==0) for dt in mdates.num2date(days)]
axC.fill_between(days, *axC.get_ylim(), where=weekends, alpha=.1)
axC.set_xlim(xmin, xmax) # set limits back to default values
axV.fill_between(days, *axV.get_ylim(), where=weekends, alpha=.1)
axV.set_xlim(xmin, xmax) # set limits back to default values

fig.legend()
fig.show()

#### Export / Import

Export

In [ ]:
#df_apple_daily_quaC.to_csv('apple_daily_quaC.csv', index=True)
#df_apple_daily_quaV.to_csv('apple_daily_quaV.csv', index=True)
#df_apple_daily_proC.to_csv('apple_daily_proC.csv', index=True)
#df_apple_daily_proV.to_csv('apple_daily_proV.csv', index=True)

Import

In [ ]:
#df_apple_daily_quaC = pd.read_csv (r'Path.csv')
#df_apple_daily_quaV = pd.read_csv (r'Path.csv')
#df_apple_daily_proC = pd.read_csv (r'Path.csv')
#df_apple_daily_proV = pd.read_csv (r'Path.csv')

***
### Air BNB (ABNB)

Let's analyse Air BNB trades behaviour in front of the trading conditions.


#### Request

In [ ]:
request_daily_abnb = tick_trade_conditions.TickTradeConditionsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_daily,
    sampling = duration.Duration(seconds = 1 * 60 * 60 * 24)
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick apple_bars service
        service = tick_trade_conditions_service.TickTradeConditionsServiceStub(channel)
        
        # process the tick apple_bars request
        abnb_daily = []
        my_metadata = [('authorization', token)]
        for stat in service.TickTradeConditions(request=request_daily_abnb, metadata=my_metadata):
            abnb_daily.append(stat)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Percentage of Trading Condition - Daily

##### Percentage based on trades counts

DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in abnb_daily]
d = {'Date': dates}
df_abnb_daily_proC= pd.DataFrame(data = d)

In [ ]:
for daily in  abnb_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_abnb_daily_proC:
            df_abnb_daily_proC.iat[df_abnb_daily_proC.index[df_abnb_daily_proC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_abnb_daily_proC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].percentage * 100
        else:
            df_abnb_daily_proC[str(daily.data[index].value)] = 0.0
            df_abnb_daily_proC.iat[df_abnb_daily_proC.index[df_abnb_daily_proC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_abnb_daily_proC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].percentage * 100
            
df_abnb_daily_proC = df_abnb_daily_proC.set_index('Date')          

In [ ]:
df_abnb_daily_proC = df_abnb_daily_proC[(df_abnb_daily_proC.T != 0.).any()]
df_abnb_daily_proC.style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_abnb_daily_proC.max()), cmap="Blues")

##### Percentage based on trades volumes

DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in abnb_daily]
d = {'Date': dates}
df_abnb_daily_proV= pd.DataFrame(data = d)

In [ ]:
for daily in  abnb_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_abnb_daily_proV:
            df_abnb_daily_proV.iat[df_abnb_daily_proV.index[df_abnb_daily_proV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_abnb_daily_proV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume_percentage * 100
        else:
            df_abnb_daily_proV[str(daily.data[index].value)] = 0.0
            df_abnb_daily_proV.iat[df_abnb_daily_proV.index[df_abnb_daily_proV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_abnb_daily_proV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume_percentage * 100
            
df_abnb_daily_proV = df_abnb_daily_proV.set_index('Date')          

In [ ]:
df_abnb_daily_proV = df_abnb_daily_proV[(df_abnb_daily_proV.T != 0.).any()]
df_abnb_daily_proV.style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_abnb_daily_proV.max()), cmap="Blues")

##### Visualisation 

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_abnb_daily_proV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_abnb_daily_proV[col], label=lab)
        axC.plot(df_abnb_daily_proC[col])
        
fig.suptitle(('Trading Conditions Daily Proportions of Air BNB (ABNB-XNGS) for {0} days').format(my_days), fontsize=20)

axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count (%)",color="blue",fontsize=14)

axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume (%)",color="blue",fontsize=14)

# Highlight weekends based on the x-axis units
xmin, xmax = axC.get_xlim()
days = np.arange(np.floor(xmin), np.ceil(xmax)+2) # range of days in date units
weekends = [(dt.weekday()>=5)|(dt.weekday()==0) for dt in mdates.num2date(days)]
axC.fill_between(days, *axC.get_ylim(), where=weekends, alpha=.1)
axC.set_xlim(xmin, xmax) # set limits back to default values
axV.fill_between(days, *axV.get_ylim(), where=weekends, alpha=.1)
axV.set_xlim(xmin, xmax) # set limits back to default values

fig.legend()
fig.show()

#### Quantities of Trading Conditions - Daily

##### Quantities based on trades counts

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in abnb_daily]
d = {'Date': dates}
df_abnb_daily_quaC= pd.DataFrame(data = d)

In [ ]:
for daily in  abnb_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_abnb_daily_quaC:
            df_abnb_daily_quaC.iat[df_abnb_daily_quaC.index[df_abnb_daily_quaC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_abnb_daily_quaC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].count
        else:
            df_abnb_daily_quaC[str(daily.data[index].value)] = 0.0
            df_abnb_daily_quaC.iat[df_abnb_daily_quaC.index[df_abnb_daily_quaC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_abnb_daily_quaC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].count
            
df_abnb_daily_quaC = df_abnb_daily_quaC.set_index('Date')          

In [ ]:
df_abnb_daily_quaC = df_abnb_daily_quaC[(df_abnb_daily_quaC.T != 0.).any()]
df_abnb_daily_quaC.style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_abnb_daily_quaC.max()), cmap="Blues")

##### Quantities based on trades volumes

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in abnb_daily]
d = {'Date': dates}
df_abnb_daily_quaV= pd.DataFrame(data = d)

In [ ]:
for daily in  abnb_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_abnb_daily_quaV:
            df_abnb_daily_quaV.iat[df_abnb_daily_quaV.index[df_abnb_daily_quaV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_abnb_daily_quaV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume
        else:
            df_abnb_daily_quaV[str(daily.data[index].value)] = 0.0
            df_abnb_daily_quaV.iat[df_abnb_daily_quaV.index[df_abnb_daily_quaV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_abnb_daily_quaV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume
            
df_abnb_daily_quaV = df_abnb_daily_quaV.set_index('Date')          

In [ ]:
df_abnb_daily_quaV = df_abnb_daily_quaV[(df_abnb_daily_quaV.T != 0.).any()]
df_abnb_daily_quaV.style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_abnb_daily_quaV.max()), cmap="Blues")

##### Visualisation

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_abnb_daily_quaV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_abnb_daily_quaV[col], label=lab)
        axC.plot(df_abnb_daily_quaC[col])
        
fig.suptitle('Trading Conditions Daily Quantities of Air BNB (ABNB-XNGS) 30 days', fontsize=20)

axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count",color="blue",fontsize=14)

axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume",color="blue",fontsize=14)

# Highlight weekends based on the x-axis units
xmin, xmax = axC.get_xlim()
days = np.arange(np.floor(xmin), np.ceil(xmax)+2) # range of days in date units
weekends = [(dt.weekday()>=5)|(dt.weekday()==0) for dt in mdates.num2date(days)]
axC.fill_between(days, *axC.get_ylim(), where=weekends, alpha=.1)
axC.set_xlim(xmin, xmax) # set limits back to default values
axV.fill_between(days, *axV.get_ylim(), where=weekends, alpha=.1)
axV.set_xlim(xmin, xmax) # set limits back to default values

fig.legend()
fig.show()

#### Export / Import

Export

In [ ]:
#df_abnb_daily_quaC.to_csv('abnb_daily_quaC.csv', index=True)
#df_abnb_daily_quaV.to_csv('abnb_daily_quaV.csv', index=True)
#df_abnb_daily_proC.to_csv('abnb_daily_proC.csv', index=True)
#df_abnb_daily_proV.to_csv('abnb_daily_proV.csv', index=True)

Import

In [ ]:
#df_abnb_daily_quaC = pd.read_csv (r'Path.csv')
#df_abnb_daily_quaV = pd.read_csv (r'Path.csv')
#df_abnb_daily_proC = pd.read_csv (r'Path.csv')
#df_abnb_daily_proV = pd.read_csv (r'Path.csv')

### DocuSign (Docu)
Let's analyse Docusign trades behaviour in front of the trading conditions.

#### Request

In [ ]:
request_docu_daily = tick_trade_conditions.TickTradeConditionsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_daily,
    sampling = duration.Duration(seconds = 1 * 60 * 60 * 24),
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick bars service
        service = tick_trade_conditions_service.TickTradeConditionsServiceStub(channel)
        
        # process the tick bars request
        docu_daily = []
        my_metadata = [('authorization', token)]
        for stat in service.TickTradeConditions(request=request_docu_daily, metadata=my_metadata):
            docu_daily.append(stat)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Percentage of Trading Condition - Daily

##### Percentage based on trades counts

DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in docu_daily]
d = {'Date': dates}
df_docu_daily_proC= pd.DataFrame(data = d)

In [ ]:
for daily in  docu_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_docu_daily_proC:
            df_docu_daily_proC.iat[df_docu_daily_proC.index[df_docu_daily_proC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_docu_daily_proC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].percentage * 100
        else:
            df_docu_daily_proC[str(daily.data[index].value)] = 0.0
            df_docu_daily_proC.iat[df_docu_daily_proC.index[df_docu_daily_proC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_docu_daily_proC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].percentage * 100
            
df_docu_daily_proC = df_docu_daily_proC.set_index('Date')          

In [ ]:
df_docu_daily_proC = df_docu_daily_proC[(df_docu_daily_proC.T != 0.).any()]
df_docu_daily_proC.style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_docu_daily_proC.max()), cmap="Blues")

##### Percentage based on trades volumes

DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in docu_daily]
d = {'Date': dates}
df_docu_daily_proV= pd.DataFrame(data = d)

In [ ]:
for daily in  docu_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_docu_daily_proV:
            df_docu_daily_proV.iat[df_docu_daily_proV.index[df_docu_daily_proV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_docu_daily_proV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume_percentage * 100
        else:
            df_docu_daily_proV[str(daily.data[index].value)] = 0.0
            df_docu_daily_proV.iat[df_docu_daily_proV.index[df_docu_daily_proV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_docu_daily_proV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume_percentage * 100
            
df_docu_daily_proV = df_docu_daily_proV.set_index('Date')          

In [ ]:

df_docu_daily_proV = df_docu_daily_proV[(df_docu_daily_proV.T != 0.).any()]
df_docu_daily_proV.style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_docu_daily_proV.max()), cmap="Blues")

##### Visualisation 

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_docu_daily_proV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_docu_daily_proV[col], label=lab)
        axC.plot(df_docu_daily_proC[col])
        
fig.suptitle(('Trading Conditions Daily Proportions of docu (AAPL-XNGS) for {0} days').format(my_days), fontsize=20)

axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count (%)",color="blue",fontsize=14)

axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume (%)",color="blue",fontsize=14)

# Highlight weekends based on the x-axis units
xmin, xmax = axC.get_xlim()
days = np.arange(np.floor(xmin), np.ceil(xmax)+2) # range of days in date units
weekends = [(dt.weekday()>=5)|(dt.weekday()==0) for dt in mdates.num2date(days)]
axC.fill_between(days, *axC.get_ylim(), where=weekends, alpha=.1)
axC.set_xlim(xmin, xmax) # set limits back to default values
axV.fill_between(days, *axV.get_ylim(), where=weekends, alpha=.1)
axV.set_xlim(xmin, xmax) # set limits back to default values

fig.legend()
fig.show()

#### Quantities of Trading Conditions - Daily

##### Quantities based on trades counts

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in docu_daily]
d = {'Date': dates}
df_docu_daily_quaC= pd.DataFrame(data = d)

In [ ]:
for daily in  docu_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_docu_daily_quaC:
            df_docu_daily_quaC.iat[df_docu_daily_quaC.index[df_docu_daily_quaC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_docu_daily_quaC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].count
        else:
            df_docu_daily_quaC[str(daily.data[index].value)] = 0.0
            df_docu_daily_quaC.iat[df_docu_daily_quaC.index[df_docu_daily_quaC['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_docu_daily_quaC.columns.get_loc(str(daily.data[index].value))] = daily.data[index].count
            
df_docu_daily_quaC = df_docu_daily_quaC.set_index('Date')          

In [ ]:
df_docu_daily_quaC = df_docu_daily_quaC[(df_docu_daily_quaC.T != 0.).any()]
df_docu_daily_quaC.style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_docu_daily_quaC.max()), cmap="Blues")

##### Quantities based on trades volumes

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in docu_daily]
d = {'Date': dates}
df_docu_daily_quaV= pd.DataFrame(data = d)

In [ ]:
for daily in  docu_daily:
    for index in range(len(daily.data)):
        if str(daily.data[index].value) in df_docu_daily_quaV:
            df_docu_daily_quaV.iat[df_docu_daily_quaV.index[df_docu_daily_quaV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_docu_daily_quaV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume
        else:
            df_docu_daily_quaV[str(daily.data[index].value)] = 0.0
            df_docu_daily_quaV.iat[df_docu_daily_quaV.index[df_docu_daily_quaV['Date'] == datetime.fromtimestamp(daily.time_stamp.seconds)].tolist()[0], df_docu_daily_quaV.columns.get_loc(str(daily.data[index].value))] = daily.data[index].volume
            
df_docu_daily_quaV = df_docu_daily_quaV.set_index('Date')          

In [ ]:
df_docu_daily_quaV = df_docu_daily_quaV[(df_docu_daily_quaV.T != 0.).any()]
df_docu_daily_quaV.style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_docu_daily_quaV.max()), cmap="Blues")

##### Visualisation

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_docu_daily_quaV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_docu_daily_quaV[col], label=lab)
        axC.plot(df_docu_daily_quaC[col])
        
fig.suptitle('Trading Conditions Daily Quantities of docu (AAPL-XNGS) 100 days', fontsize=20)

axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count",color="blue",fontsize=14)

axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume",color="blue",fontsize=14)

# Highlight weekends based on the x-axis units
xmin, xmax = axC.get_xlim()
days = np.arange(np.floor(xmin), np.ceil(xmax)+2) # range of days in date units
weekends = [(dt.weekday()>=5)|(dt.weekday()==0) for dt in mdates.num2date(days)]
axC.fill_between(days, *axC.get_ylim(), where=weekends, alpha=.1)
axC.set_xlim(xmin, xmax) # set limits back to default values
axV.fill_between(days, *axV.get_ylim(), where=weekends, alpha=.1)
axV.set_xlim(xmin, xmax) # set limits back to default values

fig.legend()
fig.show()

#### Export / Import

Export

In [ ]:
#df_docu_daily_quaC.to_csv('docu_daily_quaC.csv', index=True)
#df_docu_daily_quaV.to_csv('docu_daily_quaV.csv', index=True)
#df_docu_daily_proC.to_csv('docu_daily_proC.csv', index=True)
#df_docu_daily_proV.to_csv('docu_daily_proV.csv', index=True)

Import

In [ ]:
#df_docu_daily_quaC = pd.read_csv (r'Path.csv')
#df_docu_daily_quaV = pd.read_csv (r'Path.csv')
#df_docu_daily_proC = pd.read_csv (r'Path.csv')
#df_docu_daily_proV = pd.read_csv (r'Path.csv')

***

## V - Intraday Study

In [ ]:
today = datetime.today()

my_intervals_days = 0 # last x days for the study
my_sampling = duration.Duration(seconds = 1 * 60 * 5) # time interval for the intraday study in sec

#start = today - timedelta(days=my_intervals)
start = datetime(year = 2022, month = 4, day = 18)
end = start + timedelta(days=my_intervals_days)

date_interval_intraday = dateinterval.DateInterval(
    start_date = date.Date(year = start.year, month = start.month, day = start.day), 
    end_date = date.Date(year = end.year, month = end.month, day = end.day)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval_intraday = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 10, minutes = 30, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 23, minutes = 59, seconds = 59)
)

my_constraints_intraday = constraints.Constraints(
    date_intervals = [date_interval_intraday],
    #time_intervals = [time_interval_intraday],
    excluded_days = [dayofweek.SUNDAY, dayofweek.SATURDAY]
)

***
### Apple (AAPL)

#### Request

In [ ]:
request_intraday_apple = tick_trade_conditions.TickTradeConditionsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_intraday,
    sampling = my_sampling
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick apple_bars service
        service = tick_trade_conditions_service.TickTradeConditionsServiceStub(channel)
        
        # process the tick apple_bars request
        apple_intradays = []
        my_metadata = [('authorization', token)]
        for stat in service.TickTradeConditions(request=request_intraday_apple, metadata=my_metadata):
            apple_intradays.append(stat)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Percentage of Trading Condition - Intraday

##### Percentage based on Trades counts
DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in apple_intradays]
d_apple_intraday_proC = {'Date': dates}
df_apple_intraday_proC= pd.DataFrame(data = d_apple_intraday_proC)

In [ ]:
for intraday in  apple_intradays:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_apple_intraday_proC:
            df_apple_intraday_proC.iat[df_apple_intraday_proC.index[df_apple_intraday_proC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_apple_intraday_proC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].percentage * 100
        else:
            df_apple_intraday_proC[str(intraday.data[index].value)] = 0.0
            df_apple_intraday_proC.iat[df_apple_intraday_proC.index[df_apple_intraday_proC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_apple_intraday_proC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].percentage * 100
            
df_apple_intraday_proC = df_apple_intraday_proC.set_index('Date')          

In [ ]:
df_apple_intraday_proC = df_apple_intraday_proC[(df_apple_intraday_proC.T != 0.).any()]
df_apple_intraday_proC.head(10).style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_apple_intraday_proC.max()), cmap="Blues")

##### Percentage based on trades volumes

DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in apple_intradays]
d_apple_intraday_proV = {'Date': dates}
df_apple_intraday_proV= pd.DataFrame(data = d_apple_intraday_proV)

In [ ]:
for intraday in  apple_intradays:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_apple_intraday_proV:
            df_apple_intraday_proV.iat[df_apple_intraday_proV.index[df_apple_intraday_proV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_apple_intraday_proV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume_percentage * 100
        else:
            df_apple_intraday_proV[str(intraday.data[index].value)] = 0.0
            df_apple_intraday_proV.iat[df_apple_intraday_proV.index[df_apple_intraday_proV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_apple_intraday_proV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume_percentage * 100
            
df_apple_intraday_proV = df_apple_intraday_proV.set_index('Date')          

In [ ]:
df_apple_intraday_proV = df_apple_intraday_proV[(df_apple_intraday_proV.T != 0.).any()]
df_apple_intraday_proV.head(10).style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_apple_intraday_proV.max()), cmap="Blues")

##### Visualisation 

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_apple_intraday_proV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_apple_intraday_proV[col], label=lab)
        axC.plot(df_apple_intraday_proC[col])
        
axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count (%)",color="blue",fontsize=14) 
axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume (%)",color="blue",fontsize=14)
       


fig.suptitle('Trading Conditions Intraday Proportions of Apple (AAPL-XNGS) for intradays data', fontsize=20)
fig.legend()
plt.show()

#### Quantities of Trading Conditions - Intraday

##### Quantities based on trades counts

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in apple_intradays]
d_apple_intraday_quaC = {'Date': dates}
df_apple_intraday_quaC= pd.DataFrame(data = d_apple_intraday_quaC)

In [ ]:
for intraday in  apple_intradays:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_apple_intraday_quaC:
            df_apple_intraday_quaC.iat[df_apple_intraday_quaC.index[df_apple_intraday_quaC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_apple_intraday_quaC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].count
        else:
            df_apple_intraday_quaC[str(intraday.data[index].value)] = 0.0
            df_apple_intraday_quaC.iat[df_apple_intraday_quaC.index[df_apple_intraday_quaC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_apple_intraday_quaC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].count
            
df_apple_intraday_quaC = df_apple_intraday_quaC.set_index('Date')          

In [ ]:
df_apple_intraday_quaC = df_apple_intraday_quaC[(df_apple_intraday_quaC.T != 0.).any()]
df_apple_intraday_quaC.head(10).style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_apple_intraday_quaC.max()), cmap="Blues")

##### Quantities based on trades volumes

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in apple_intradays]
d_apple_intraday_quaV = {'Date': dates}
df_apple_intraday_quaV= pd.DataFrame(data = d_apple_intraday_quaV)

In [ ]:
for intraday in  apple_intradays:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_apple_intraday_quaV:
            df_apple_intraday_quaV.iat[df_apple_intraday_quaV.index[df_apple_intraday_quaV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_apple_intraday_quaV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume
        else:
            df_apple_intraday_quaV[str(intraday.data[index].value)] = 0.0
            df_apple_intraday_quaV.iat[df_apple_intraday_quaV.index[df_apple_intraday_quaV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_apple_intraday_quaV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume
            
df_apple_intraday_quaV = df_apple_intraday_quaV.set_index('Date')          

In [ ]:
df_apple_intraday_quaV = df_apple_intraday_quaV[(df_apple_intraday_quaV.T != 0.).any()]
df_apple_intraday_quaV.head(10).style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_apple_intraday_quaV.max()), cmap="Blues")

##### Visualisation

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_apple_intraday_quaV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_apple_intraday_quaV[col], label=lab)
        axC.plot(df_apple_intraday_quaC[col])

axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count",color="blue",fontsize=14)
axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume",color="blue",fontsize=14)

fig.suptitle('Trading Conditions Intraday Quantities of Apple (AAPL-XNGS) for intradays', fontsize=20)
fig.legend(fontsize=14)
plt.show()

#### Export / Import

Export

In [ ]:
#df_apple_intraday_quaC.to_csv('apple_intradays_quaC.csv', index=True)
#df_apple_intraday_quaV.to_csv('apple_intradays_quaV.csv', index=True)
#df_apple_intraday_proC.to_csv('apple_intradays_proC.csv', index=True)
#df_apple_intraday_proV.to_csv('apple_intradays_proV.csv', index=True)

Import

In [ ]:
#df_apple_intraday_quaC = pd.read_csv (r'Path.csv')
#df_apple_intraday_quaV = pd.read_csv (r'Path.csv')
#df_apple_intradays_proC = pd.read_csv (r'Path.csv')
#df_apple_intradays_proV = pd.read_csv (r'Path.csv')

***

### Air BNB (ABNB)


#### Request

In [ ]:
request_intraday_abnb = tick_trade_conditions.TickTradeConditionsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_intraday,
    sampling = my_sampling
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick apple_bars service
        service = tick_trade_conditions_service.TickTradeConditionsServiceStub(channel)
        
        # process the tick apple_bars request
        abnb_intraday = []
        my_metadata = [('authorization', token)]
        for stat in service.TickTradeConditions(request=request_intraday_abnb, metadata=my_metadata):
            abnb_intraday.append(stat)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Percentage of Trading Condition - Intraday

##### Percentage based on Trades counts
DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in abnb_intraday]
d_abnb_intraday_proC = {'Date': dates}
df_abnb_intraday_proC= pd.DataFrame(data = d_abnb_intraday_proC)

In [ ]:
for intraday in  abnb_intraday:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_abnb_intraday_proC:
            df_abnb_intraday_proC.iat[df_abnb_intraday_proC.index[df_abnb_intraday_proC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_abnb_intraday_proC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].percentage * 100
        else:
            df_abnb_intraday_proC[str(intraday.data[index].value)] = 0.0
            df_abnb_intraday_proC.iat[df_abnb_intraday_proC.index[df_abnb_intraday_proC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_abnb_intraday_proC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].percentage * 100
            
df_abnb_intraday_proC = df_abnb_intraday_proC.set_index('Date')          

In [ ]:
df_abnb_intraday_proC = df_abnb_intraday_proC[(df_abnb_intraday_proC.T != 0.).any()]
df_abnb_intraday_proC.head(10).style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_abnb_intraday_proC.max()), cmap="Blues")

##### Percentage based on trades volumes

DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in abnb_intraday]
d_abnb_intraday_proC = {'Date': dates}
df_abnb_intraday_proV= pd.DataFrame(data = d_abnb_intraday_proC)

In [ ]:
for intraday in  abnb_intraday:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_abnb_intraday_proV:
            df_abnb_intraday_proV.iat[df_abnb_intraday_proV.index[df_abnb_intraday_proV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_abnb_intraday_proV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume_percentage * 100
        else:
            df_abnb_intraday_proV[str(intraday.data[index].value)] = 0.0
            df_abnb_intraday_proV.iat[df_abnb_intraday_proV.index[df_abnb_intraday_proV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_abnb_intraday_proV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume_percentage * 100
            
df_abnb_intraday_proV = df_abnb_intraday_proV.set_index('Date')          

In [ ]:
df_abnb_intraday_proV = df_abnb_intraday_proV[(df_abnb_intraday_proV.T != 0.).any()]
df_abnb_intraday_proV.head(10).style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_abnb_intraday_proV.max()), cmap="Blues")

##### Visualisation 

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_abnb_intraday_proV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_abnb_intraday_proV[col], label=lab)
        axC.plot(df_abnb_intraday_proC[col])

axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume (%)",color="blue",fontsize=14)
axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count (%)",color="blue",fontsize=14) 

       


fig.suptitle('Trading Conditions Intraday Proportions of Air BNB (ABNB-XNGS) for intradays data', fontsize=20)
fig.legend()
plt.show()

#### Quantities of Trading Conditions - Intraday

##### Quantities based on trades counts

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in abnb_intraday]
d = {'Date': dates}
df_abnb_intraday_quaC= pd.DataFrame(data = d)

In [ ]:
for intraday in  abnb_intraday:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_abnb_intraday_quaC:
            df_abnb_intraday_quaC.iat[df_abnb_intraday_quaC.index[df_abnb_intraday_quaC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_abnb_intraday_quaC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].count
        else:
            df_abnb_intraday_quaC[str(intraday.data[index].value)] = 0.0
            df_abnb_intraday_quaC.iat[df_abnb_intraday_quaC.index[df_abnb_intraday_quaC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_abnb_intraday_quaC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].count
            
df_abnb_intraday_quaC = df_abnb_intraday_quaC.set_index('Date')          

In [ ]:
df_abnb_intraday_quaC = df_abnb_intraday_quaC[(df_abnb_intraday_quaC.T != 0.).any()]
df_abnb_intraday_quaC.head(10).style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_abnb_intraday_quaC.max()), cmap="Blues")

##### Quantities based on trades volumes

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in abnb_intraday]
d = {'Date': dates}
df_abnb_intraday_quaV= pd.DataFrame(data = d)

In [ ]:
for intraday in  abnb_intraday:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_abnb_intraday_quaV:
            df_abnb_intraday_quaV.iat[df_abnb_intraday_quaV.index[df_abnb_intraday_quaV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_abnb_intraday_quaV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume
        else:
            df_abnb_intraday_quaV[str(intraday.data[index].value)] = 0.0
            df_abnb_intraday_quaV.iat[df_abnb_intraday_quaV.index[df_abnb_intraday_quaV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_abnb_intraday_quaV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume
            
df_abnb_intraday_quaV = df_abnb_intraday_quaV.set_index('Date')          

In [ ]:
df_abnb_intraday_quaV = df_abnb_intraday_quaV[(df_abnb_intraday_quaV.T != 0.).any()]
df_abnb_intraday_quaV.head(10).style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_abnb_intraday_quaV.max()), cmap="Blues")

##### Visualisation

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_abnb_intraday_quaV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_abnb_intraday_quaV[col], label=lab)
        axC.plot(df_abnb_intraday_quaC[col])

axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count",color="blue",fontsize=14)
axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume",color="blue",fontsize=14)

fig.suptitle('Trading Conditions Intraday Quantities of Air BNB (ABNB-XNGS) for intradays data', fontsize=20)
fig.legend(fontsize=14)
plt.show()

#### Export / Import

Export

In [ ]:
#df_abnb_intraday_quaC.to_csv('abnb_intraday_quaC.csv', index=True)
#df_abnb_intraday_quaV.to_csv('abnb_intraday_quaV.csv', index=True)
#df_abnb_intraday_proC.to_csv('abnb_intraday_proC.csv', index=True)
#df_abnb_intraday_proV.to_csv('abnb_intraday_proV.csv', index=True)

Import

In [ ]:
#df_abnb_intraday_quaC = pd.read_csv (r'Path.csv')
#df_abnb_intraday_quaV = pd.read_csv (r'Path.csv')
#df_abnb_intradays_proC = pd.read_csv (r'Path.csv')
#df_abnb_intradays_proV = pd.read_csv (r'Path.csv')

***
### Docusign (DOCU)

#### Request

In [ ]:
request_intraday_docu = tick_trade_conditions.TickTradeConditionsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_intraday,
    sampling = my_sampling
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick bars service
        service = tick_trade_conditions_service.TickTradeConditionsServiceStub(channel)
        
        # process the tick bars request
        docu_intradays = []
        my_metadata = [('authorization', token)]
        for stat in service.TickTradeConditions(request=request_intraday_docu, metadata=my_metadata):
            docu_intradays.append(stat)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Percentage of Trading Condition - Intraday

##### Percentage based on Trades counts
DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in docu_intradays]
d_docu_intraday_proC = {'Date': dates}
df_docu_intraday_proC= pd.DataFrame(data = d_docu_intraday_proC)

In [ ]:
for intraday in  docu_intradays:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_docu_intraday_proC:
            df_docu_intraday_proC.iat[df_docu_intraday_proC.index[df_docu_intraday_proC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_docu_intraday_proC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].percentage * 100
        else:
            df_docu_intraday_proC[str(intraday.data[index].value)] = 0.0
            df_docu_intraday_proC.iat[df_docu_intraday_proC.index[df_docu_intraday_proC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_docu_intraday_proC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].percentage * 100
            
df_docu_intraday_proC = df_docu_intraday_proC.set_index('Date')          

In [ ]:
df_docu_intraday_proC = df_docu_intraday_proC[(df_docu_intraday_proC.T != 0.).any()]
df_docu_intraday_proC.head(10).style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_docu_intraday_proC.max()), cmap="Blues")

##### Percentage based on trades volumes

DataFrame Creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in docu_intradays]
d_docu_intraday_proV = {'Date': dates}
df_docu_intraday_proV= pd.DataFrame(data = d_docu_intraday_proV)

In [ ]:
for intraday in  docu_intradays:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_docu_intraday_proV:
            df_docu_intraday_proV.iat[df_docu_intraday_proV.index[df_docu_intraday_proV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_docu_intraday_proV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume_percentage * 100
        else:
            df_docu_intraday_proV[str(intraday.data[index].value)] = 0.0
            df_docu_intraday_proV.iat[df_docu_intraday_proV.index[df_docu_intraday_proV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_docu_intraday_proV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume_percentage * 100
            
df_docu_intraday_proV = df_docu_intraday_proV.set_index('Date')          

In [ ]:
df_docu_intraday_proV = df_docu_intraday_proV[(df_docu_intraday_proV.T != 0.).any()]
df_docu_intraday_proV.head(10).style.format(precision = 5).background_gradient(axis=None, vmin=0, vmax=max(df_docu_intraday_proV.max()), cmap="Blues")

##### Visualisation 

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_docu_intraday_proV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_docu_intraday_proV[col], label=lab)
        axC.plot(df_docu_intraday_proC[col])
        
axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count (%)",color="blue",fontsize=14) 
axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume (%)",color="blue",fontsize=14)
       


fig.suptitle('Trading Conditions Intraday Proportions of docu (AAPL-XNGS) for intradays data', fontsize=20)
fig.legend()
plt.show()

#### Quantities of Trading Conditions - Intraday

##### Quantities based on trades counts

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in docu_intradays]
d_docu_intraday_quaC = {'Date': dates}
df_docu_intraday_quaC= pd.DataFrame(data = d_docu_intraday_quaC)

In [ ]:
for intraday in  docu_intradays:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_docu_intraday_quaC:
            df_docu_intraday_quaC.iat[df_docu_intraday_quaC.index[df_docu_intraday_quaC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_docu_intraday_quaC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].count
        else:
            df_docu_intraday_quaC[str(intraday.data[index].value)] = 0.0
            df_docu_intraday_quaC.iat[df_docu_intraday_quaC.index[df_docu_intraday_quaC['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_docu_intraday_quaC.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].count
            
df_docu_intraday_quaC = df_docu_intraday_quaC.set_index('Date')          

In [ ]:
df_docu_intraday_quaC = df_docu_intraday_quaC[(df_docu_intraday_quaC.T != 0.).any()]
df_docu_intraday_quaC.head(10).style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_docu_intraday_quaC.max()), cmap="Blues")

##### Quantities based on trades volumes

DataFrame creation

In [ ]:
dates = [datetime.fromtimestamp(t.time_stamp.seconds) for t in docu_intradays]
d_docu_intraday_quaV = {'Date': dates}
df_docu_intraday_quaV= pd.DataFrame(data = d_docu_intraday_quaV)

In [ ]:
for intraday in  docu_intradays:
    for index in range(len(intraday.data)):
        if str(intraday.data[index].value) in df_docu_intraday_quaV:
            df_docu_intraday_quaV.iat[df_docu_intraday_quaV.index[df_docu_intraday_quaV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_docu_intraday_quaV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume
        else:
            df_docu_intraday_quaV[str(intraday.data[index].value)] = 0.0
            df_docu_intraday_quaV.iat[df_docu_intraday_quaV.index[df_docu_intraday_quaV['Date'] == datetime.fromtimestamp(intraday.time_stamp.seconds)].tolist()[0], df_docu_intraday_quaV.columns.get_loc(str(intraday.data[index].value))] = intraday.data[index].volume
            
df_docu_intraday_quaV = df_docu_intraday_quaV.set_index('Date')          

In [ ]:
df_docu_intraday_quaV = df_docu_intraday_quaV[(df_docu_intraday_quaV.T != 0.).any()]
df_docu_intraday_quaV.head(10).style.format(precision = 0).background_gradient(axis=None, vmin=0, vmax=max(df_docu_intraday_quaV.max()), cmap="Blues")

##### Visualisation

In [ ]:
fig,(axC,axV) = plt.subplots(2,1,figsize=(25,10))
for col in df_docu_intraday_quaV.columns:
    if not col == 'Date':
        lab = df_xngs.loc[df_xngs.Value==col, 'Description'].values[0]
        axV.plot(df_docu_intraday_quaV[col], label=lab)
        axC.plot(df_docu_intraday_quaC[col])

axC.set_xlabel("Date",fontsize=14)
axC.set_ylabel("Count",color="blue",fontsize=14)
axV.set_xlabel("Date",fontsize=14)
axV.set_ylabel("Volume",color="blue",fontsize=14)

fig.suptitle('Trading Conditions Intraday Quantities of docu (AAPL-XNGS) for intradays', fontsize=20)
fig.legend(fontsize=14)
plt.show()

#### Export / Import

Export

In [ ]:
#df_docu_intraday_quaC.to_csv('docu_intradays_quaC.csv', index=True)
#df_docu_intraday_quaV.to_csv('docu_intradays_quaV.csv', index=True)
#df_docu_intraday_proC.to_csv('docu_intradays_proC.csv', index=True)
#df_docu_intraday_proV.to_csv('docu_intradays_proV.csv', index=True)

Import

In [ ]:
#df_docu_intraday_quaC = pd.read_csv (r'Path.csv')
#df_docu_intraday_quaV = pd.read_csv (r'Path.csv')
#df_docu_intradays_proC = pd.read_csv (r'Path.csv')
#df_docu_intradays_proV = pd.read_csv (r'Path.csv')

***
## IV - VWAP
Let's study the vwap indicators of our three tickers for a random market day.

We are going to focus on 6 different combinations in a time interval of one day:
-	All the trading conditions: everything is kept.
-	The three main trading conditions (Odd Lot Trade, Regular Sale, Intermarket Sweep): we keep the main three trading conditions and reject all the others.
-	All the others: the main three trading conditions are rejected and the rest is kept.
-	Individual Analysis: Odd Lot Trade condition, Regular Sale condition and Intermarket Sweep condition


### Parameters

In [ ]:
# create time intervals (we are using Google date format)
date_interval_vwap = dateinterval.DateInterval(
    start_date = date.Date(year = 2022, month = 1, day = 5), 
    end_date = date.Date(year = 2022, month = 1, day = 5)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval_vwap = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 14, minutes = 30, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 21, minutes = 0, seconds = 0)
)

In [ ]:
# generate constraints based on the previous time selection
my_constraints_vwap = constraints.Constraints(
    date_intervals = [date_interval_vwap],
    time_intervals = [time_interval_vwap],
)

### Apple

#### Request

In [ ]:
# generate the tick vwap request
request_apple_vwap = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_vwap
)

request_apple_vwap_069 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_vwap,
    accept = ['0','6','9'] # keep only the trading conditions with these values
)

request_apple_vwap_no069 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_vwap,
    reject = ['0','6','9'] # keep all the trading conditions except these values
)

request_apple_vwap_0 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_vwap,
    accept = ['0']
)

request_apple_vwap_6 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_vwap,
    accept = ['6']
)

request_apple_vwap_9 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_vwap,
    accept = ['9']
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_apple, trades_apple, vwaps_apple = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_apple_vwap, metadata=metadata):
            time_stamps_apple.append(v.time_stamp)
            trades_apple.append(v.trade)
            vwaps_apple.append(v.vwap)
            
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_apple_069, trades_apple_069, vwaps_apple_069 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_apple_vwap_069, metadata=metadata):
            time_stamps_apple_069.append(v.time_stamp)
            trades_apple_069.append(v.trade)
            vwaps_apple_069.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_apple_no069, trades_apple_no069, vwaps_apple_no069 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_apple_vwap_no069, metadata=metadata):
            time_stamps_apple_no069.append(v.time_stamp)
            trades_apple_no069.append(v.trade)
            vwaps_apple_no069.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_apple_0, trades_apple_0, vwaps_apple_0 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_apple_vwap_0, metadata=metadata):
            time_stamps_apple_0.append(v.time_stamp)
            trades_apple_0.append(v.trade)
            vwaps_apple_0.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_apple_6, trades_apple_6, vwaps_apple_6 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_apple_vwap_6, metadata=metadata):
            time_stamps_apple_6.append(v.time_stamp)
            trades_apple_6.append(v.trade)
            vwaps_apple_6.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_apple_9, trades_apple_9, vwaps_apple_9 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_apple_vwap_9, metadata=metadata):
            time_stamps_apple_9.append(v.time_stamp)
            trades_apple_9.append(v.trade)
            vwaps_apple_9.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Dataframes Creation

In [ ]:
# create the vwap dataframe
dates_apple=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_apple]
d_apple_vwap = {'Date': dates_apple, 'Trade': trades_apple, 'Vwap': vwaps_apple}
df_apple_vwap_all = pd.DataFrame(data=d_apple_vwap)
df_apple_vwap_all.head(10)

In [ ]:
# create the vwap dataframe
dates_apple_069=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_apple_069]
d_apple_vwap_069 = {'Date': dates_apple_069, 'Trade': trades_apple_069, 'Vwap': vwaps_apple_069}
df_apple_vwap_069 = pd.DataFrame(data=d_apple_vwap_069)
df_apple_vwap_069.head(10)

In [ ]:
# create the vwap dataframe
dates_apple_no069=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_apple_no069]
d_apple_vwap_no069 = {'Date': dates_apple_no069, 'Trade': trades_apple_no069, 'Vwap': vwaps_apple_no069}
df_apple_vwap_no069 = pd.DataFrame(data=d_apple_vwap_no069)
df_apple_vwap_no069.head(10)

In [ ]:
# create the vwap dataframe
dates_apple_0=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_apple_0]
d_apple_vwap_0 = {'Date': dates_apple_0, 'Trade': trades_apple_0, 'Vwap': vwaps_apple_0}
df_apple_vwap_0 = pd.DataFrame(data=d_apple_vwap_0)
df_apple_vwap_0.head(10)

In [ ]:
# create the vwap dataframe
dates_apple_6=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_apple_6]
d_apple_vwap_6 = {'Date': dates_apple_6, 'Trade': trades_apple_6, 'Vwap': vwaps_apple_6}
df_apple_vwap_6 = pd.DataFrame(data=d_apple_vwap_6)
df_apple_vwap_6.head(10)

In [ ]:
# create the vwap dataframe
dates_apple_9=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_apple_9]
d_apple_vwap_9 = {'Date': dates_apple_9, 'Trade': trades_apple_9, 'Vwap': vwaps_apple_9}
df_apple_vwap_9 = pd.DataFrame(data=d_apple_vwap_9)
df_apple_vwap_9.head(10)

#### Visualisation

In [ ]:
# create figure and axis objects with subplots()
fig, ax = plt.subplots(1,1,figsize=(25,13))

ax.plot('Date', 'Trade', data=df_apple_vwap_all, marker='', color='blue', linewidth=1, label="Trades")

ax.plot('Date', 'Vwap', data=df_apple_vwap_all, marker='', color='orange', linewidth=1, label="VWAP")
ax.plot('Date', 'Vwap', data=df_apple_vwap_069, marker='', color='red', linewidth=1, label="VWAP_069")
ax.plot('Date', 'Vwap', data=df_apple_vwap_no069, marker='', color='green', linewidth=1, label="VWAP_no069")
ax.plot('Date', 'Vwap', data=df_apple_vwap_0, marker='', color='black', linewidth=1, label="VWAP_0")
ax.plot('Date', 'Vwap', data=df_apple_vwap_6, marker='', color='purple', linewidth=1, label="VWAP_6")
ax.plot('Date', 'Vwap', data=df_apple_vwap_9, marker='', color='yellow', linewidth=1, label="VWAP_9")

ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="black",fontsize=14)
plt.legend()
plt.title('Tick Vwap and Trades for {0}-{1}'.format('AAPL', 'XNGS'),fontsize=20)
plt.show()

### ABNB

#### Request

In [ ]:
# generate the tick vwap request
request_abnb_vwap = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_vwap
)

request_abnb_vwap_069 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_vwap,
    accept = ['0','6','9'] # keep only the trading conditions with these values
)

request_abnb_vwap_no069 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_vwap,
    reject = ['0','6','9'] # keep all the trading conditions except these values
)

request_abnb_vwap_0 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_vwap,
    accept = ['0']
)

request_abnb_vwap_6 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_vwap,
    accept = ['6']
)

request_abnb_vwap_9 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_vwap,
    accept = ['9']
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_abnb, trades_abnb, vwaps_abnb = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_abnb_vwap, metadata=metadata):
            time_stamps_abnb.append(v.time_stamp)
            trades_abnb.append(v.trade)
            vwaps_abnb.append(v.vwap)
            
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_abnb_069, trades_abnb_069, vwaps_abnb_069 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_abnb_vwap_069, metadata=metadata):
            time_stamps_abnb_069.append(v.time_stamp)
            trades_abnb_069.append(v.trade)
            vwaps_abnb_069.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_abnb_no069, trades_abnb_no069, vwaps_abnb_no069 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_abnb_vwap_no069, metadata=metadata):
            time_stamps_abnb_no069.append(v.time_stamp)
            trades_abnb_no069.append(v.trade)
            vwaps_abnb_no069.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_abnb_0, trades_abnb_0, vwaps_abnb_0 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_abnb_vwap_0, metadata=metadata):
            time_stamps_abnb_0.append(v.time_stamp)
            trades_abnb_0.append(v.trade)
            vwaps_abnb_0.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_abnb_6, trades_abnb_6, vwaps_abnb_6 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_abnb_vwap_6, metadata=metadata):
            time_stamps_abnb_6.append(v.time_stamp)
            trades_abnb_6.append(v.trade)
            vwaps_abnb_6.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_abnb_9, trades_abnb_9, vwaps_abnb_9 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_abnb_vwap_9, metadata=metadata):
            time_stamps_abnb_9.append(v.time_stamp)
            trades_abnb_9.append(v.trade)
            vwaps_abnb_9.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Dataframes Creation

In [ ]:
# create the vwap dataframe
dates_abnb=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_abnb]
d_abnb_vwap = {'Date': dates_abnb, 'Trade': trades_abnb, 'Vwap': vwaps_abnb}
df_abnb_vwap_all = pd.DataFrame(data=d_abnb_vwap)
df_abnb_vwap_all.head(10)

In [ ]:
# create the vwap dataframe
dates_abnb_069=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_abnb_069]
d_vwap_abnb_069 = {'Date': dates_abnb_069, 'Trade': trades_abnb_069, 'Vwap': vwaps_abnb_069}
df_abnb_vwap_069 = pd.DataFrame(data=d_vwap_abnb_069)
df_abnb_vwap_069.head(10)

In [ ]:
# create the vwap dataframe
dates_abnb_no069=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_abnb_no069]
d_vwap_abnb_no069 = {'Date': dates_abnb_no069, 'Trade': trades_abnb_no069, 'Vwap': vwaps_abnb_no069}
df_abnb_vwap_no069 = pd.DataFrame(data=d_vwap_abnb_no069)
df_abnb_vwap_no069.head(10)

In [ ]:
# create the vwap dataframe
dates_abnb_0=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_abnb_0]
d_vwap_abnb_0 = {'Date': dates_abnb_0, 'Trade': trades_abnb_0, 'Vwap': vwaps_abnb_0}
df_abnb_vwap_0 = pd.DataFrame(data=d_vwap_abnb_0)
df_abnb_vwap_0.head(10)

In [ ]:
# create the vwap dataframe
dates_abnb_6=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_abnb_6]
d_vwap_abnb_6 = {'Date': dates_abnb_6, 'Trade': trades_abnb_6, 'Vwap': vwaps_abnb_6}
df_abnb_vwap_6 = pd.DataFrame(data=d_vwap_abnb_6)
df_abnb_vwap_6.head(10)

In [ ]:
# create the vwap dataframe
dates_abnb_9=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_abnb_9]
d_vwap_abnb_9 = {'Date': dates_abnb_9, 'Trade': trades_abnb_9, 'Vwap': vwaps_abnb_9}
df_abnb_vwap_9 = pd.DataFrame(data=d_vwap_abnb_9)
df_abnb_vwap_9.head(10)

#### Visualisation

In [ ]:
# create figure and axis objects with subplots()
fig, ax = plt.subplots(1,1,figsize=(25,13))

ax.plot('Date', 'Trade', data=df_abnb_vwap_all, marker='', color='blue', linewidth=1, label="Trades")

ax.plot('Date', 'Vwap', data=df_abnb_vwap_all, marker='', color='orange', linewidth=1, label="VWAP")
ax.plot('Date', 'Vwap', data=df_abnb_vwap_069, marker='', color='red', linewidth=1, label="VWAP_069")
ax.plot('Date', 'Vwap', data=df_abnb_vwap_no069, marker='', color='green', linewidth=1, label="VWAP_no069")
ax.plot('Date', 'Vwap', data=df_abnb_vwap_0, marker='', color='black', linewidth=1, label="VWAP_0")
ax.plot('Date', 'Vwap', data=df_abnb_vwap_6, marker='', color='purple', linewidth=1, label="VWAP_6")
ax.plot('Date', 'Vwap', data=df_abnb_vwap_9, marker='', color='yellow', linewidth=1, label="VWAP_9")

ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="black",fontsize=14)
plt.legend()
plt.title('Tick Vwap and Trades for {0}-{1}'.format('ABNB', 'XNGS'),fontsize=20)
plt.show()

### Docusign

#### Request

In [ ]:
# generate the tick vwap request
request_docu_vwap = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_vwap
)

request_docu_vwap_069 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_vwap,
    accept = ['0','6','9'] # keep only the trading conditions with these values
)

request_docu_vwap_no069 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_vwap,
    reject = ['0','6','9'] # keep all the trading conditions except these values
)

request_docu_vwap_0 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_vwap,
    accept = ['0']
)

request_docu_vwap_6 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_vwap,
    accept = ['6']
)

request_docu_vwap_9 = tick_vwap.TickVwapRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_vwap,
    accept = ['9']
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_docu, trades_docu, vwaps_docu = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_docu_vwap, metadata=metadata):
            time_stamps_docu.append(v.time_stamp)
            trades_docu.append(v.trade)
            vwaps_docu.append(v.vwap)
            
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_docu_069, trades_docu_069, vwaps_docu_069 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_docu_vwap_069, metadata=metadata):
            time_stamps_docu_069.append(v.time_stamp)
            trades_docu_069.append(v.trade)
            vwaps_docu_069.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_docu_no069, trades_docu_no069, vwaps_docu_no069 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_docu_vwap_no069, metadata=metadata):
            time_stamps_docu_no069.append(v.time_stamp)
            trades_docu_no069.append(v.trade)
            vwaps_docu_no069.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_docu_0, trades_docu_0, vwaps_docu_0 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_docu_vwap_0, metadata=metadata):
            time_stamps_docu_0.append(v.time_stamp)
            trades_docu_0.append(v.trade)
            vwaps_docu_0.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_docu_6, trades_docu_6, vwaps_docu_6 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_docu_vwap_6, metadata=metadata):
            time_stamps_docu_6.append(v.time_stamp)
            trades_docu_6.append(v.trade)
            vwaps_docu_6.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick vwap service
        service = tick_vwap_service.TickVwapServiceStub(channel)
        
        # process the tick vwap request
        time_stamps_docu_9, trades_docu_9, vwaps_docu_9 = [], [], []
        metadata = [('authorization', token)]
        for v in service.TickVwap(request=request_docu_vwap_9, metadata=metadata):
            time_stamps_docu_9.append(v.time_stamp)
            trades_docu_9.append(v.trade)
            vwaps_docu_9.append(v.vwap)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Dataframes Creation

In [ ]:
# create the vwap dataframe
dates_docu=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_docu]
d_docu_vwap = {'Date': dates_docu, 'Trade': trades_docu, 'Vwap': vwaps_docu}
df_docu_vwap_all = pd.DataFrame(data=d_docu_vwap)
df_docu_vwap_all.head(10)

In [ ]:
# create the vwap dataframe
dates_docu_069=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_docu_069]
d_docu_vwap_069 = {'Date': dates_docu_069, 'Trade': trades_docu_069, 'Vwap': vwaps_docu_069}
df_docu_vwap_069 = pd.DataFrame(data=d_docu_vwap_069)
df_docu_vwap_069.head(10)

In [ ]:
# create the vwap dataframe
dates_docu_no069=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_docu_no069]
d_docu_vwap_no069 = {'Date': dates_docu_no069, 'Trade': trades_docu_no069, 'Vwap': vwaps_docu_no069}
df_docu_vwap_no069 = pd.DataFrame(data=d_docu_vwap_no069)
df_docu_vwap_no069.head(10)

In [ ]:
# create the vwap dataframe
dates_docu_0=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_docu_0]
d_docu_vwap_0 = {'Date': dates_docu_0, 'Trade': trades_docu_0, 'Vwap': vwaps_docu_0}
df_docu_vwap_0 = pd.DataFrame(data=d_docu_vwap_0)
df_docu_vwap_0.head(10)

In [ ]:
# create the vwap dataframe
dates_docu_6=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_docu_6]
d_docu_vwap_6 = {'Date': dates_docu_6, 'Trade': trades_docu_6, 'Vwap': vwaps_docu_6}
df_docu_vwap_6 = pd.DataFrame(data=d_docu_vwap_6)
df_docu_vwap_6.head(10)

In [ ]:
# create the vwap dataframe
dates_docu_9=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps_docu_9]
d_docu_vwap_9 = {'Date': dates_docu_9, 'Trade': trades_docu_9, 'Vwap': vwaps_docu_9}
df_docu_vwap_9 = pd.DataFrame(data=d_docu_vwap_9)
df_docu_vwap_9.head(10)

#### Visualisation

In [ ]:
# create figure and axis objects with subplots()
fig, ax = plt.subplots(1,1,figsize=(25,13))

ax.plot('Date', 'Trade', data=df_docu_vwap_all, marker='', color='blue', linewidth=1, label="Trades")

ax.plot('Date', 'Vwap', data=df_docu_vwap_all, marker='', color='orange', linewidth=1, label="VWAP")
ax.plot('Date', 'Vwap', data=df_docu_vwap_069, marker='', color='red', linewidth=1, label="VWAP_069")
ax.plot('Date', 'Vwap', data=df_docu_vwap_no069, marker='', color='green', linewidth=1, label="VWAP_no069")
ax.plot('Date', 'Vwap', data=df_docu_vwap_0, marker='', color='black', linewidth=1, label="VWAP_0")
ax.plot('Date', 'Vwap', data=df_docu_vwap_6, marker='', color='purple', linewidth=1, label="VWAP_6")
ax.plot('Date', 'Vwap', data=df_docu_vwap_9, marker='', color='yellow', linewidth=1, label="VWAP_9")

ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="black",fontsize=14)
plt.legend()
plt.title('Tick Vwap and Trades for {0}-{1}'.format('DOCU', 'XNGS'),fontsize=20)
plt.show()

***
## V - Candlestick/Bars graph
We are now going to see on our chosen equities, candlestick grpah visualisation differs with or without some trading conditions.

### Parameters

In [ ]:
# set the bar duration
sampling = 1 * 60 * 10 # 10 min bars graph

# set the bar calculation field
field = tick_bars.BAR_PRICE_TRADE 

In [ ]:
# create time intervals (we are using Google date format)
date_interval_bars = dateinterval.DateInterval(
    start_date = date.Date(year = 2022, month = 1, day = 5), 
    end_date = date.Date(year = 2022, month = 1, day = 5)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval_bars = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 14, minutes = 30, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 21, minutes = 0, seconds = 0)
)

In [ ]:
# generate constraints based on the previous time selection
my_constraints_bars = constraints.Constraints(
    date_intervals = [date_interval_bars],
    time_intervals = [time_interval_bars],
)

In [ ]:
def highlight(s):
    if s.Open > s.Close:
        return ['background-color: red; opacity: 0.5; color:white '] * len(s)
    else:
        return ['background-color: green; opacity: 0.5; color:white'] * len(s)

### Apple


#### Request

In [ ]:
# generate the tick_bars request
request_apple_bars = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field
)

request_apple_bars_069 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['0','6','9']
)

request_apple_bars_no069 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    reject = ['0','6','9']
)

request_apple_bars_0 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['0']
)

request_apple_bars_6 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['6']
)

request_apple_bars_9 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'AAPL'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['9']
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick apple_bars service
        service = tick_bars_service.TickBarsServiceStub(channel)
        
        # process the tick apple_bars requests
        apple_bars_all = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_apple_bars, metadata=metadata):
                apple_bars_all.append(bar)
        
        apple_bars_069 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_apple_bars_069, metadata=metadata):
                apple_bars_069.append(bar)
                
        apple_bars_no069 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_apple_bars_no069, metadata=metadata):
                apple_bars_no069.append(bar)
        
        apple_bars_0 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_apple_bars_0, metadata=metadata):
                apple_bars_0.append(bar)
        
        apple_bars_6 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_apple_bars_6, metadata=metadata):
                apple_bars_6.append(bar)
        
        apple_bars_9 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_apple_bars_9, metadata=metadata):
                apple_bars_9.append(bar)
        
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Datesframes Creation

In [ ]:
# prepare the dataframe content
dates_apple_bars_all=[datetime.fromtimestamp(b.time_stamp.seconds) for b in apple_bars_all]
opens_apple_bars_all = [b.open for b in apple_bars_all]
highs_apple_bars_all = [b.high for b in apple_bars_all]
lows_apple_bars_all = [b.low for b in apple_bars_all]
closes_apple_bars_all = [b.close for b in apple_bars_all]
volumes_apple_bars_all = [ts.volume for ts in apple_bars_all]
counts_apple_bars_all = [ts.count for ts in apple_bars_all]
bars_apple_bars_all = [ts.vwap for ts in apple_bars_all]

# create a pandas dataframe with: dates, apple_bars and ticks count used for each bar
d_apple_bars_all = {'Date': dates_apple_bars_all, 'Open': opens_apple_bars_all, 'High': highs_apple_bars_all, 'Low' : lows_apple_bars_all,'Close': closes_apple_bars_all, 'Volume': volumes_apple_bars_all, 'Count': counts_apple_bars_all, 'Vwap': bars_apple_bars_all}
df_apple_bars_all = pd.DataFrame(data=d_apple_bars_all)
df_apple_bars_all = df_apple_bars_all.set_index('Date')
df_apple_bars_all.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_apple_bars_069=[datetime.fromtimestamp(b.time_stamp.seconds) for b in apple_bars_069]
opens_apple_bars_069 = [b.open for b in apple_bars_069]
highs_apple_bars_069 = [b.high for b in apple_bars_069]
lows_apple_bars_069 = [b.low for b in apple_bars_069]
closes_apple_bars_069 = [b.close for b in apple_bars_069]
volumes_apple_bars_069 = [ts.volume for ts in apple_bars_069]
counts_apple_bars_069 = [ts.count for ts in apple_bars_069]
bars_apple_bars_069 = [ts.vwap for ts in apple_bars_069]

# create a pandas dataframe with: dates, apple_bars and ticks count used for each bar
d_apple_bars_069 = {'Date': dates_apple_bars_069, 'Open': opens_apple_bars_069, 'High': highs_apple_bars_069, 'Low' : lows_apple_bars_069,'Close': closes_apple_bars_069, 'Volume': volumes_apple_bars_069, 'Count': counts_apple_bars_069, 'Vwap': bars_apple_bars_069}
df_apple_bars_069 = pd.DataFrame(data=d_apple_bars_069)
df_apple_bars_069 = df_apple_bars_069.set_index('Date')
df_apple_bars_069.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_apple_bars_no069=[datetime.fromtimestamp(b.time_stamp.seconds) for b in apple_bars_no069]
opens_apple_bars_no069 = [b.open for b in apple_bars_no069]
highs_apple_bars_no069 = [b.high for b in apple_bars_no069]
lows_apple_bars_no069 = [b.low for b in apple_bars_no069]
closes_apple_bars_no069 = [b.close for b in apple_bars_no069]
volumes_apple_bars_no069 = [ts.volume for ts in apple_bars_no069]
counts_apple_bars_no069 = [ts.count for ts in apple_bars_no069]
bars_apple_bars_no069 = [ts.vwap for ts in apple_bars_no069]

# create a pandas dataframe with: dates, apple_bars and ticks count used for each bar
d_apple_bars_no069 = {'Date': dates_apple_bars_no069, 'Open': opens_apple_bars_no069, 'High': highs_apple_bars_no069, 'Low' : lows_apple_bars_no069,'Close': closes_apple_bars_no069, 'Volume': volumes_apple_bars_no069, 'Count': counts_apple_bars_no069, 'Vwap': bars_apple_bars_no069}
df_apple_bars_no069 = pd.DataFrame(data=d_apple_bars_no069)
df_apple_bars_no069 = df_apple_bars_no069.set_index('Date')
df_apple_bars_no069.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_apple_bars_0=[datetime.fromtimestamp(b.time_stamp.seconds) for b in apple_bars_0]
opens_apple_bars_0 = [b.open for b in apple_bars_0]
highs_apple_bars_0 = [b.high for b in apple_bars_0]
lows_apple_bars_0 = [b.low for b in apple_bars_0]
closes_apple_bars_0 = [b.close for b in apple_bars_0]
volumes_apple_bars_0 = [ts.volume for ts in apple_bars_0]
counts_apple_bars_0 = [ts.count for ts in apple_bars_0]
bars_apple_bars_0 = [ts.vwap for ts in apple_bars_0]

# create a pandas dataframe with: dates, apple_bars and ticks count used for each bar
d_apple_bars_0 = {'Date': dates_apple_bars_0, 'Open': opens_apple_bars_0, 'High': highs_apple_bars_0, 'Low' : lows_apple_bars_0,'Close': closes_apple_bars_0, 'Volume': volumes_apple_bars_0, 'Count': counts_apple_bars_0, 'Vwap': bars_apple_bars_0}
df_apple_bars_0 = pd.DataFrame(data=d_apple_bars_0)
df_apple_bars_0 = df_apple_bars_0.set_index('Date')
df_apple_bars_0.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_apple_bars_6=[datetime.fromtimestamp(b.time_stamp.seconds) for b in apple_bars_6]
opens_apple_bars_6 = [b.open for b in apple_bars_6]
highs_apple_bars_6 = [b.high for b in apple_bars_6]
lows_apple_bars_6 = [b.low for b in apple_bars_6]
closes_apple_bars_6 = [b.close for b in apple_bars_6]
volumes_apple_bars_6 = [ts.volume for ts in apple_bars_6]
counts_apple_bars_6 = [ts.count for ts in apple_bars_6]
bars_apple_bars_6 = [ts.vwap for ts in apple_bars_6]

# create a pandas dataframe with: dates, apple_bars and ticks count used for each bar
d_apple_bars_6 = {'Date': dates_apple_bars_6, 'Open': opens_apple_bars_6, 'High': highs_apple_bars_6, 'Low' : lows_apple_bars_6,'Close': closes_apple_bars_6, 'Volume': volumes_apple_bars_6, 'Count': counts_apple_bars_6, 'Vwap': bars_apple_bars_6}
df_apple_bars_6 = pd.DataFrame(data=d_apple_bars_6)
df_apple_bars_6 = df_apple_bars_6.set_index('Date')
df_apple_bars_6.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_apple_bars_9=[datetime.fromtimestamp(b.time_stamp.seconds) for b in apple_bars_9]
opens_apple_bars_9 = [b.open for b in apple_bars_9]
highs_apple_bars_9 = [b.high for b in apple_bars_9]
lows_apple_bars_9 = [b.low for b in apple_bars_9]
closes_apple_bars_9 = [b.close for b in apple_bars_9]
volumes_apple_bars_9 = [ts.volume for ts in apple_bars_9]
counts_apple_bars_9 = [ts.count for ts in apple_bars_9]
bars_apple_bars_9 = [ts.vwap for ts in apple_bars_9]

# create a pandas dataframe with: dates, apple_bars and ticks count used for each bar
d_apple_bars_9 = {'Date': dates_apple_bars_9, 'Open': opens_apple_bars_9, 'High': highs_apple_bars_9, 'Low' : lows_apple_bars_9,'Close': closes_apple_bars_9, 'Volume': volumes_apple_bars_9, 'Count': counts_apple_bars_9, 'Vwap': bars_apple_bars_9}
df_apple_bars_9 = pd.DataFrame(data=d_apple_bars_9)
df_apple_bars_9 = df_apple_bars_9.set_index('Date')
df_apple_bars_9.head(5).style.apply(highlight, axis=1)

#### Visualisation

In [ ]:
mpf.plot(df_apple_bars_all, figratio=(40,15),
         type = 'candle', 
         title=('All Trading Conditions'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_apple_bars_069, figratio=(40,15),
         type = 'candle', 
         title=('The Three Main Trading Conditions'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_apple_bars_no069, figratio=(40,15),
         type = 'candle', 
         title=('All the Others'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_apple_bars_0, figratio=(40,15),
         type = 'candle', 
         title=('Regular Trades'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_apple_bars_6, figratio=(40,15),
         type = 'candle', 
         title=('Intermarket Sweep'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_apple_bars_9, figratio=(40,15),
         type = 'candle', 
         title=('Odd Lot Trades'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        ) 

### Air BNB


#### Request

In [ ]:
# generate the tick _bars request
request_abnb_bars = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field
)

request_abnb_bars_069 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['0','6','9']
)

request_abnb_bars_no069 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    reject = ['0','6','9']
)

request_abnb_bars_0 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['0']
)

request_abnb_bars_6 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['6']
)

request_abnb_bars_9 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'ABNB'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['9']
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick bars service
        service = tick_bars_service.TickBarsServiceStub(channel)
        
        # process the tick bars requests
        abnb_bars_all = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_abnb_bars, metadata=metadata):
                abnb_bars_all.append(bar)
        
        abnb_bars_069 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_abnb_bars_069, metadata=metadata):
                abnb_bars_069.append(bar)
                
        abnb_bars_no069 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_abnb_bars_no069, metadata=metadata):
                abnb_bars_no069.append(bar)
        
        abnb_bars_0 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_abnb_bars_0, metadata=metadata):
                abnb_bars_0.append(bar)
        
        abnb_bars_6 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_abnb_bars_6, metadata=metadata):
                abnb_bars_6.append(bar)
        
        abnb_bars_9 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_abnb_bars_9, metadata=metadata):
                abnb_bars_9.append(bar)
        
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Datesframes Creation

In [ ]:
# prepare the dataframe content
dates_abnb_bars_all=[datetime.fromtimestamp(b.time_stamp.seconds) for b in abnb_bars_all]
opens_abnb_bars_all = [b.open for b in abnb_bars_all]
highs_abnb_bars_all = [b.high for b in abnb_bars_all]
lows_abnb_bars_all = [b.low for b in abnb_bars_all]
closes_abnb_bars_all = [b.close for b in abnb_bars_all]
volumes_abnb_bars_all = [ts.volume for ts in abnb_bars_all]
counts_abnb_bars_all = [ts.count for ts in abnb_bars_all]
bars_abnb_bars_all = [ts.vwap for ts in abnb_bars_all]

# create a pandas dataframe with: dates, bars and ticks count used for each bar
d_abnb_bars_all = {'Date': dates_abnb_bars_all, 'Open': opens_abnb_bars_all, 'High': highs_abnb_bars_all, 'Low' : lows_abnb_bars_all,'Close': closes_abnb_bars_all, 'Volume': volumes_abnb_bars_all, 'Count': counts_abnb_bars_all, 'Vwap': bars_abnb_bars_all}
df_abnb_bars_all = pd.DataFrame(data=d_abnb_bars_all)
df_abnb_bars_all = df_abnb_bars_all.set_index('Date')
df_abnb_bars_all.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_abnb_bars_069=[datetime.fromtimestamp(b.time_stamp.seconds) for b in abnb_bars_069]
opens_abnb_bars_069 = [b.open for b in abnb_bars_069]
highs_abnb_bars_069 = [b.high for b in abnb_bars_069]
lows_abnb_bars_069 = [b.low for b in abnb_bars_069]
closes_abnb_bars_069 = [b.close for b in abnb_bars_069]
volumes_abnb_bars_069 = [ts.volume for ts in abnb_bars_069]
counts_abnb_bars_069 = [ts.count for ts in abnb_bars_069]
bars_abnb_bars_069 = [ts.vwap for ts in abnb_bars_069]

# create a pandas dataframe with: dates, abnb_bars and ticks count used for each bar
d_abnb_bars_069 = {'Date': dates_abnb_bars_069, 'Open': opens_abnb_bars_069, 'High': highs_abnb_bars_069, 'Low' : lows_abnb_bars_069,'Close': closes_abnb_bars_069, 'Volume': volumes_abnb_bars_069, 'Count': counts_abnb_bars_069, 'Vwap': bars_abnb_bars_069}
df_abnb_bars_069 = pd.DataFrame(data=d_abnb_bars_069)
df_abnb_bars_069 = df_abnb_bars_069.set_index('Date')
df_abnb_bars_069.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_abnb_bars_no069=[datetime.fromtimestamp(b.time_stamp.seconds) for b in abnb_bars_no069]
opens_abnb_bars_no069 = [b.open for b in abnb_bars_no069]
highs_abnb_bars_no069 = [b.high for b in abnb_bars_no069]
lows_abnb_bars_no069 = [b.low for b in abnb_bars_no069]
closes_abnb_bars_no069 = [b.close for b in abnb_bars_no069]
volumes_abnb_bars_no069 = [ts.volume for ts in abnb_bars_no069]
counts_abnb_bars_no069 = [ts.count for ts in abnb_bars_no069]
bars_abnb_bars_no069 = [ts.vwap for ts in abnb_bars_no069]

# create a pandas dataframe with: dates, abnb_bars and ticks count used for each bar
d_abnb_bars_no069 = {'Date': dates_abnb_bars_no069, 'Open': opens_abnb_bars_no069, 'High': highs_abnb_bars_no069, 'Low' : lows_abnb_bars_no069,'Close': closes_abnb_bars_no069, 'Volume': volumes_abnb_bars_no069, 'Count': counts_abnb_bars_no069, 'Vwap': bars_abnb_bars_no069}
df_abnb_bars_no069 = pd.DataFrame(data=d_abnb_bars_no069)
df_abnb_bars_no069 = df_abnb_bars_no069.set_index('Date')
df_abnb_bars_no069.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_abnb_bars_0=[datetime.fromtimestamp(b.time_stamp.seconds) for b in abnb_bars_0]
opens_abnb_bars_0 = [b.open for b in abnb_bars_0]
highs_abnb_bars_0 = [b.high for b in abnb_bars_0]
lows_abnb_bars_0 = [b.low for b in abnb_bars_0]
closes_abnb_bars_0 = [b.close for b in abnb_bars_0]
volumes_abnb_bars_0 = [ts.volume for ts in abnb_bars_0]
counts_abnb_bars_0 = [ts.count for ts in abnb_bars_0]
bars_abnb_bars_0 = [ts.vwap for ts in abnb_bars_0]

# create a pandas dataframe with: dates, abnb_bars and ticks count used for each bar
d_abnb_bars_0 = {'Date': dates_abnb_bars_0, 'Open': opens_abnb_bars_0, 'High': highs_abnb_bars_0, 'Low' : lows_abnb_bars_0,'Close': closes_abnb_bars_0, 'Volume': volumes_abnb_bars_0, 'Count': counts_abnb_bars_0, 'Vwap': bars_abnb_bars_0}
df_abnb_bars_0 = pd.DataFrame(data=d_abnb_bars_0)
df_abnb_bars_0 = df_abnb_bars_0.set_index('Date')
df_abnb_bars_0.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_abnb_bars_6=[datetime.fromtimestamp(b.time_stamp.seconds) for b in abnb_bars_6]
opens_abnb_bars_6 = [b.open for b in abnb_bars_6]
highs_abnb_bars_6 = [b.high for b in abnb_bars_6]
lows_abnb_bars_6 = [b.low for b in abnb_bars_6]
closes_abnb_bars_6 = [b.close for b in abnb_bars_6]
volumes_abnb_bars_6 = [ts.volume for ts in abnb_bars_6]
counts_abnb_bars_6 = [ts.count for ts in abnb_bars_6]
bars_abnb_bars_6 = [ts.vwap for ts in abnb_bars_6]

# create a pandas dataframe with: dates, abnb_bars and ticks count used for each bar
d_abnb_bars_6 = {'Date': dates_abnb_bars_6, 'Open': opens_abnb_bars_6, 'High': highs_abnb_bars_6, 'Low' : lows_abnb_bars_6,'Close': closes_abnb_bars_6, 'Volume': volumes_abnb_bars_6, 'Count': counts_abnb_bars_6, 'Vwap': bars_abnb_bars_6}
df_abnb_bars_6 = pd.DataFrame(data=d_abnb_bars_6)
df_abnb_bars_6 = df_abnb_bars_6.set_index('Date')
df_abnb_bars_6.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_abnb_bars_9=[datetime.fromtimestamp(b.time_stamp.seconds) for b in abnb_bars_9]
opens_abnb_bars_9 = [b.open for b in abnb_bars_9]
highs_abnb_bars_9 = [b.high for b in abnb_bars_9]
lows_abnb_bars_9 = [b.low for b in abnb_bars_9]
closes_abnb_bars_9 = [b.close for b in abnb_bars_9]
volumes_abnb_bars_9 = [ts.volume for ts in abnb_bars_9]
counts_abnb_bars_9 = [ts.count for ts in abnb_bars_9]
bars_abnb_bars_9 = [ts.vwap for ts in abnb_bars_9]

# create a pandas dataframe with: dates, abnb_bars and ticks count used for each bar
d_abnb_bars_9 = {'Date': dates_abnb_bars_9, 'Open': opens_abnb_bars_9, 'High': highs_abnb_bars_9, 'Low' : lows_abnb_bars_9,'Close': closes_abnb_bars_9, 'Volume': volumes_abnb_bars_9, 'Count': counts_abnb_bars_9, 'Vwap': bars_abnb_bars_9}
df_abnb_bars_9 = pd.DataFrame(data=d_abnb_bars_9)
df_abnb_bars_9 = df_abnb_bars_9.set_index('Date')
df_abnb_bars_9.head(5).style.apply(highlight, axis=1)

#### Visualisation

In [ ]:
mpf.plot(df_abnb_bars_all, figratio=(40,15),
         type = 'candle', 
         title=('All Trading Conditions'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_abnb_bars_069, figratio=(40,15),
         type = 'candle', 
         title=('The Three Main Trading Conditions'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_abnb_bars_no069, figratio=(40,15),
         type = 'candle', 
         title=('All the Others'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_abnb_bars_0, figratio=(40,15),
         type = 'candle', 
         title=('Regular Trades'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_abnb_bars_6, figratio=(40,15),
         type = 'candle', 
         title=('Intermarket Sweep'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_abnb_bars_9, figratio=(40,15),
         type = 'candle', 
         title=('Odd Lot Trades'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )   

### DocuSign


#### Request

In [ ]:
# generate the tick_bars request
request_docu_bars = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field
)

request_docu_bars_069 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['0','6','9']
)

request_docu_bars_no069 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    reject = ['0','6','9']
)

request_docu_bars_0 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['0']
)

request_docu_bars_6 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['6']
)

request_docu_bars_9 = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = 'XNGS', ticker = 'DOCU'),
    constraints = my_constraints_bars,
    sampling = duration.Duration(seconds = sampling),
    field = field,
    accept = ['9']
)

In [ ]:
%%time
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick bars service
        service = tick_bars_service.TickBarsServiceStub(channel)
        
        # process the tick bars requests
        docu_bars_all = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_docu_bars, metadata=metadata):
                docu_bars_all.append(bar)
        
        docu_bars_069 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_docu_bars_069, metadata=metadata):
                docu_bars_069.append(bar)
                
        docu_bars_no069 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_docu_bars_no069, metadata=metadata):
                docu_bars_no069.append(bar)
        
        docu_bars_0 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_docu_bars_0, metadata=metadata):
                docu_bars_0.append(bar)
        
        docu_bars_6 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_docu_bars_6, metadata=metadata):
                docu_bars_6.append(bar)
        
        docu_bars_9 = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request_docu_bars_9, metadata=metadata):
                docu_bars_9.append(bar)
        
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

#### Datesframes Creation

In [ ]:
# prepare the dataframe content
dates_docu_bars_all=[datetime.fromtimestamp(b.time_stamp.seconds) for b in docu_bars_all]
opens_docu_bars_all = [b.open for b in docu_bars_all]
highs_docu_bars_all = [b.high for b in docu_bars_all]
lows_docu_bars_all = [b.low for b in docu_bars_all]
closes_docu_bars_all = [b.close for b in docu_bars_all]
volumes_docu_bars_all = [ts.volume for ts in docu_bars_all]
counts_docu_bars_all = [ts.count for ts in docu_bars_all]
bars_docu_bars_all = [ts.vwap for ts in docu_bars_all]

# create a pandas dataframe with: dates, docu_bars and ticks count used for each bar
d_docu_bars_all = {'Date': dates_docu_bars_all, 'Open': opens_docu_bars_all, 'High': highs_docu_bars_all, 'Low' : lows_docu_bars_all,'Close': closes_docu_bars_all, 'Volume': volumes_docu_bars_all, 'Count': counts_docu_bars_all, 'Vwap': bars_docu_bars_all}
df_docu_bars_all = pd.DataFrame(data=d_docu_bars_all)
df_docu_bars_all = df_docu_bars_all.set_index('Date')
df_docu_bars_all.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_docu_bars_069=[datetime.fromtimestamp(b.time_stamp.seconds) for b in docu_bars_069]
opens_docu_bars_069 = [b.open for b in docu_bars_069]
highs_docu_bars_069 = [b.high for b in docu_bars_069]
lows_docu_bars_069 = [b.low for b in docu_bars_069]
closes_docu_bars_069 = [b.close for b in docu_bars_069]
volumes_docu_bars_069 = [ts.volume for ts in docu_bars_069]
counts_docu_bars_069 = [ts.count for ts in docu_bars_069]
bars_docu_bars_069 = [ts.vwap for ts in docu_bars_069]

# create a pandas dataframe with: dates, docu_bars and ticks count used for each bar
d_docu_bars_069 = {'Date': dates_docu_bars_069, 'Open': opens_docu_bars_069, 'High': highs_docu_bars_069, 'Low' : lows_docu_bars_069,'Close': closes_docu_bars_069, 'Volume': volumes_docu_bars_069, 'Count': counts_docu_bars_069, 'Vwap': bars_docu_bars_069}
df_docu_bars_069 = pd.DataFrame(data=d_docu_bars_069)
df_docu_bars_069 = df_docu_bars_069.set_index('Date')
df_docu_bars_069.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_docu_bars_no069=[datetime.fromtimestamp(b.time_stamp.seconds) for b in docu_bars_no069]
opens_docu_bars_no069 = [b.open for b in docu_bars_no069]
highs_docu_bars_no069 = [b.high for b in docu_bars_no069]
lows_docu_bars_no069 = [b.low for b in docu_bars_no069]
closes_docu_bars_no069 = [b.close for b in docu_bars_no069]
volumes_docu_bars_no069 = [ts.volume for ts in docu_bars_no069]
counts_docu_bars_no069 = [ts.count for ts in docu_bars_no069]
bars_docu_bars_no069 = [ts.vwap for ts in docu_bars_no069]

# create a pandas dataframe with: dates, docu_bars and ticks count used for each bar
d_docu_bars_no069 = {'Date': dates_docu_bars_no069, 'Open': opens_docu_bars_no069, 'High': highs_docu_bars_no069, 'Low' : lows_docu_bars_no069,'Close': closes_docu_bars_no069, 'Volume': volumes_docu_bars_no069, 'Count': counts_docu_bars_no069, 'Vwap': bars_docu_bars_no069}
df_docu_bars_no069 = pd.DataFrame(data=d_docu_bars_no069)
df_docu_bars_no069 = df_docu_bars_no069.set_index('Date')
df_docu_bars_no069.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_docu_bars_0=[datetime.fromtimestamp(b.time_stamp.seconds) for b in docu_bars_0]
opens_docu_bars_0 = [b.open for b in docu_bars_0]
highs_docu_bars_0 = [b.high for b in docu_bars_0]
lows_docu_bars_0 = [b.low for b in docu_bars_0]
closes_docu_bars_0 = [b.close for b in docu_bars_0]
volumes_docu_bars_0 = [ts.volume for ts in docu_bars_0]
counts_docu_bars_0 = [ts.count for ts in docu_bars_0]
bars_docu_bars_0 = [ts.vwap for ts in docu_bars_0]

# create a pandas dataframe with: dates, docu_bars and ticks count used for each bar
d_docu_bars_0 = {'Date': dates_docu_bars_0, 'Open': opens_docu_bars_0, 'High': highs_docu_bars_0, 'Low' : lows_docu_bars_0,'Close': closes_docu_bars_0, 'Volume': volumes_docu_bars_0, 'Count': counts_docu_bars_0, 'Vwap': bars_docu_bars_0}
df_docu_bars_0 = pd.DataFrame(data=d_docu_bars_0)
df_docu_bars_0 = df_docu_bars_0.set_index('Date')
df_docu_bars_0.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_docu_bars_6=[datetime.fromtimestamp(b.time_stamp.seconds) for b in docu_bars_6]
opens_docu_bars_6 = [b.open for b in docu_bars_6]
highs_docu_bars_6 = [b.high for b in docu_bars_6]
lows_docu_bars_6 = [b.low for b in docu_bars_6]
closes_docu_bars_6 = [b.close for b in docu_bars_6]
volumes_docu_bars_6 = [ts.volume for ts in docu_bars_6]
counts_docu_bars_6 = [ts.count for ts in docu_bars_6]
bars_docu_bars_6 = [ts.vwap for ts in docu_bars_6]

# create a pandas dataframe with: dates, docu_bars and ticks count used for each bar
d_docu_bars_6 = {'Date': dates_docu_bars_6, 'Open': opens_docu_bars_6, 'High': highs_docu_bars_6, 'Low' : lows_docu_bars_6,'Close': closes_docu_bars_6, 'Volume': volumes_docu_bars_6, 'Count': counts_docu_bars_6, 'Vwap': bars_docu_bars_6}
df_docu_bars_6 = pd.DataFrame(data=d_docu_bars_6)
df_docu_bars_6 = df_docu_bars_6.set_index('Date')
df_docu_bars_6.head(5).style.apply(highlight, axis=1)

In [ ]:
# prepare the dataframe content
dates_docu_bars_9=[datetime.fromtimestamp(b.time_stamp.seconds) for b in docu_bars_9]
opens_docu_bars_9 = [b.open for b in docu_bars_9]
highs_docu_bars_9 = [b.high for b in docu_bars_9]
lows_docu_bars_9 = [b.low for b in docu_bars_9]
closes_docu_bars_9 = [b.close for b in docu_bars_9]
volumes_docu_bars_9 = [ts.volume for ts in docu_bars_9]
counts_docu_bars_9 = [ts.count for ts in docu_bars_9]
bars_docu_bars_9 = [ts.vwap for ts in docu_bars_9]

# create a pandas dataframe with: dates, docu_bars and ticks count used for each bar
d_docu_bars_9 = {'Date': dates_docu_bars_9, 'Open': opens_docu_bars_9, 'High': highs_docu_bars_9, 'Low' : lows_docu_bars_9,'Close': closes_docu_bars_9, 'Volume': volumes_docu_bars_9, 'Count': counts_docu_bars_9, 'Vwap': bars_docu_bars_9}
df_docu_bars_9 = pd.DataFrame(data=d_docu_bars_9)
df_docu_bars_9 = df_docu_bars_9.set_index('Date')
df_docu_bars_9.head(5).style.apply(highlight, axis=1)

#### Visualisation

In [ ]:
mpf.plot(df_docu_bars_all, figratio=(40,15),
         type = 'candle', 
         title=('ALL'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_docu_bars_069, figratio=(40,15),
         type = 'candle', 
         title=('069'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_docu_bars_no069, figratio=(40,15),
         type = 'candle', 
         title=('no069'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_docu_bars_0, figratio=(40,15),
         type = 'candle', 
         title=('0'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_docu_bars_6, figratio=(40,15),
         type = 'candle', 
         title=('6'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

mpf.plot(df_docu_bars_9, figratio=(40,15),
         type = 'candle', 
         title=('9'),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        ) 